# ch07 템플릿 메소드로 알고리즘 재정의하기

- 가끔은 약간만 수정해서 사용해야 하는 알고리즘
- 둘은 서로 다른 URL을 사용하고 전달하는 데이터도 다르다.

## 템플릿 메소드 디자인 패턴

- 핵심: 어떤 목표를 달성하기 위한 알고리즘에 대해 작업 단계(기초 작업)를 담아둔 메소드를 만드는 것
- 이 기초 작업은 별도의 메소드로 구현됨. 모든 알고리즘에 동일하게 사용하는 메소드도 있고, 서로 다른 메소드도 있다.
- 동일한 메소드는 추상 클래스로 구현하지만, 서로 다른 메소드는 알고리즘마다 별도의 클래스로 구현함

### 템플릿 메소드 구현 순서

1. 알고리즘에 대한 메소드를 만듦
2. 알고리즘 구현을 여러 메소드로 나눔
3. 모든 클래스에 동일한 메소드는 상위 클래스에서 구현하고, 특정 메소드는 상속받은 클래스에서 구현한다.

### 템플릿 메소드 디자인 패턴의 장점

- 클래스의 일부를 노출하거나 제어하게 해서 확장성이 좋다는 점
- 코드 중복을 최소화함. 비슷한 코드를 반복적으로 작성할 필요가 없음
- 알고리즘 자체는 한 곳에 위치하므로, 여러 부분에서 수정할 필요가 없음
- 코드 수정이 수월해짐. 알고리즘을 처음부터 재작성할 필요없이 클래스를 새로 만들고 필요한 메소드만 구현하면 됨

### 후크 사용

- Hook는 추상 클래스에서 정의 가능하고 실제 클래스에서 오버라이드 할 수 있는 메소드
- 후크는 하위 클래스에서 오버라이드 가능하지만 강제사항은 아님
- 하지만 실제 작업은 반드시 구현해야 하고, 구현이 빠지면 NotImplementedError 예외가 발생한다는 차이점
- 후크는 코드 중복을 피하기 위해 알고리즘을 살짝 수정할 때 사용함

## 파이썬 구현

- 여러 뉴스 사이트에서 최신 뉴스를 모아주는 수집기를 만든다고 가정
- 뉴스 사이트는 대게 RSS와 Atom 프로토콜로 뉴스를 제공함
- 이런 프로토콜은 XML을 기반으로 한 비슷한 형식으로 약간의 차이점이 존재

#### RSS Feed

```xml
<?xml version="1.0" encoding="ISO-8859-1" ?>
     <rss version="2.0">
       <channel>
         <title>A RSS example</title>
         <link>http://example.com</link>
         <description>Description of RSS example</description>
         <item>
           <title>The first news</title>
           <link>http://example.com/first</link>
           <description>Some description of the first
             news</description>
         </item>
         <item>
           <guid>urn:uuid:1225c695-cfb8-4ebb-aaaa-80da344efa6a</id>
           <title>The second news</title>
           <link>example.com/second</link>
           <description>Some description of the second
             news</description>
           <pubDate>Wed, 30 Sep 2013 13:00:00 GMT</pubDate>
    </item>
  </channel>
</rss>
```

#### Atom Feed

```xml
<?xml version="1.0" encoding="utf-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
<title>A RSS example</title>
<subtitle>Description of RSS example</subtitle>
<link href="http://example.org/"/>
<entry>
 <id>urn:uuid:1225c695-cfb8-4ebb-aaaa-80da344efa6a</id>
 <title>The first news</title>
 <link href="http://example.com/first"/>
 <summary>Some description of the second news</summary>
 <updated>2013-09-30T13:00:00Z</updated>
</entry>
</feed>
```

- 서로 다른 두 피드를 파싱하기 위해서는 별도의 알고리즘이 필요하지만, 그외 요청, 응답 얻기, 파싱한 내용 출력 등에는 동일한 코드 사용 가능
- Atom 서식을 사용하는 피드로 구글 뉴스, RSS 서식을 사용하는 피드로 야후 뉴스를 사용
- 우리의 고객은 stdout에 최신 뉴스를 출력하고 싶어한다.

### 파싱 알고리즘

1. URL을 얻고 피드 서버에 요청을 보낸다.(특별)
2. 가공되지 않은 콘텐츠를 얻는다.(동일)
3. 파싱한다.(특별)
4. 사용자에게 출력해서 보여준다.(동일)

In [13]:
import urllib2 # RSS, ATOM 피드에 http 요청을 보내기 위해서


class AbstractNewsParser(object):
    def __init__(self):
        # 클래스 인스턴스 생성 방지
        if self.__class__ is AbstractNewsParser:
            raise TypeError('abstract class cannot be instantiated')
            
    def print_top_news(self):
        """템플릿 메소드. 모든 웹사이트에서 3개의 최신 뉴스를 반환한다."""
        url = self.get_url()
        raw_content = self.get_raw_content(url)
        content = self.parse_content(raw_content)
        cropped = self.crop(content)
        
        for item in cropped:
            print('Title: ', item['title'])
            print('Content: ', item['content'])
            print('Link: ', item['link'])
            print('Published: ', item['published'])
            print('Id: ', item['id'])
            
    def get_url(self):
        raise NotImplementedError()
        
    def get_raw_content(self, url):
        return urllib2.urlopen(url).read()
    
    def parse_content(self, content):
        raise NotImplementedError
        
    def crop(self, parsed_content, max_items=3):
        return parsed_content[:max_items]

In [40]:
from xml.dom import minidom


class YahooParser(AbstractNewsParser):
    def get_url(self):
        return 'http://news.yahoo.com/rss/'
    
    def parse_content(self, raw_content):
        parsed_content = []
        dom = minidom.parseString(raw_content)
        for node in dom.getElementsByTagName('item'):
            parsed_item = {}
            try:
                parsed_item['title'] = node.getElementsByTagName('title')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['title'] = None
                
            try:
                parsed_item['content'] = \
                node.getElementsByTagName('description')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['content'] = None
                
            try:
                parsed_item['link'] = \
                node.getElementsByTagName('link')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['link'] = None
                
            try:
                parsed_item['id'] = \
                node.getElementsByTagName('guid')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['id'] = None
                
            try:
                parsed_item['published'] = \
                node.getElementsByTagName('pubDate')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['published'] = None
                
            parsed_content.append(parsed_item)
        return parsed_content

In [18]:
class GoogleParser(AbstractNewsParser):
    def get_url(self):
        return 'http://news.google.com/news/feeds?output=atom'
    
    def parse_content(self, raw_content):
        parsed_content = []
        dom = minidom.parseString(raw_content)
        
        for node in dom.getElementsByTagName('entry'):
            parsed_item = {}
            try:
                parsed_item['title'] = \
                node.getElementsByTagName('title')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['title'] = None
                
            try:
                parsed_item['content'] = \
                node.getElementsByTagName('content')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['content'] = None
                
            try:
                parsed_item['link'] = \
                node.getElementsByTagName('link')[0].getAttribute('href')
            except IndexError:
                parsed_item['link'] = None
                
            try:
                parsed_item['id'] = \
                node.getElementsByTagName('id')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['id'] = None
                
            try:
                parsed_item['published'] = \
                node.getElementsByTagName('updated')[0].childNodes[0].nodeValue
            except IndexError:
                parsed_item['published'] = None
                
            parsed_content.append(parsed_item)
        return parsed_content

In [45]:
if __name__ == '__main__':
    google = GoogleParser()
    yahoo = YahooParser()
    
    print('Google: ')
    print(google.print_top_news())
    print('')
    print('Yahoo: ')
    print(yahoo.print_top_news())

Google: 
('Title: ', u'Pope to meet prisoners at end of immigration-focused US visit - Reuters')
('Content: ', u'<table border="0" cellpadding="2" cellspacing="7" style="vertical-align:top;"><tr><td width="80" align="center" valign="top"><font style="font-size:85%;font-family:arial,sans-serif"><a href="http://news.google.com/news/url?sa=t&amp;fd=R&amp;ct2=us&amp;usg=AFQjCNGQz4gL81op7p48ZdB-FyfQbXYW3A&amp;clid=c3a7d30bb8a4878e06b80cf16b898331&amp;cid=52778958855938&amp;ei=1OUHVpjcMsOl4AKRp7iYAQ&amp;url=http://www.reuters.com/article/2015/09/27/us-pope-usa-idUSKCN0RP11X20150927"><img src="//t3.gstatic.com/images?q=tbn:ANd9GcTh4CiqzLDRlZuGK_m1Fh1ambpISqtBwEhYPvvAxtRecv3pavP37tkJyKIDbU_KFoxS5OJM6kzr" alt="" border="1" width="80" height="80"><br><font size="-2">Reuters</font></a></font></td><td valign="top" class="j"><font style="font-size:85%;font-family:arial,sans-serif"><br><div style="padding-top:0.8em;"><img alt="" height="1" width="1"></div><div class="lh"><a href="http://news.google.

## 요약

- 템플릿 메소드는 알고리즘의 기초를 정의하고 그 뒤에서 구조의 변화없이 몇몇 알고리즘 단게를 재정의하는 디자인 패턴
- 템플릿 메소드 패턴은 알고리즘을 쉽게 확장 가능하게 만들지만, 재정의는 허용된 경우에만 가능함
- 이 디자인 패턴은, 알고리즘의 동작은 일반적이지만 콘텍스트가 달라짐에 따라 구현부가 달라지는 경우에 잘 어울림